Ты можешь найти мои комментарии, обозначенные <font color='green'>зеленым</font>, <font color='gold'>желтым</font> и <font color='red'>красным</font> цветами, например:

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> В случае, если решение на отдельном шаге является полностью правильным.
</div>

<br/>

<div class="alert alert-warning">
    <h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> В случае, когда решение на отдельном шаге станет еще лучше, если внести небольшие коррективы.
</div>


<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>

    
<b>На доработку🤔:</b>
 В случае, когда решение на отдельном шаге требует существенной переработки и внесения правок. Напоминаю, что проект не может быть принят с первого раза, если ревью содержит комментарии, рекомендующие доработать шаги.
</div>
    
    
<br/>    
<div class="alert alert-info">
<h2> Комментарий студента: <a class="tocSkip"> </h2>

<b>👋:</b> В такой цветовой ячейке я прошу тебя оставлять свои комментарии. Если исправляешь проект на второй итерации и выше, не забывай пожалуйста указывать номер итерации, например, "Комментарий студента v.2".
</div> 

<br/>    
    
Увидев у тебя неточность, в первый раз я лишь укажу на ее наличие и дам тебе возможность самому найти и исправить ее. На реальной работе твой руководитель будет поступать также, и я пытаюсь подготовить тебя именно к работе датасаентистом. Но если ты пока не справишься с такой задачей - при следующей проверке я дам более точную подсказку!🤓

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Добавим необходимые методы, библиотеки для дальнейшей работы с датасетом и решении поставленной задачи.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

Откроем файл, запишем их в переменную data, проанализируем:

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Датасет на 3214 записей, можно индефицировать как 3214 клиентов, пропусков нет, чистили его насколько я помню в прошлом модуле, поэтому в предоборотке датасет не нуждается. 

### Вывод

Нам достался датасет, с которым мы когда то работали, поэтому предообработка ему не нужна.

<div class="alert alert-warning">
    <h2> Комментарий ревьюера <a class="tocSkip"></a></h2>
    
<b>Некоторые замечания и рекомендации💡:</b> С данными познакомились, проведен первичный аналитический осмотр!) Полученная информация поможет нам в дальнейшем с предобработкой данных. 
        
- Хочу тут затронуть термин мультиколлениарности, чтобы было представление о нем. В наших данных есть мультиколлинеарность в фичах calls и minutes. Желательно проверить это методом `.corr(method='spearman')`       
        
    В нашем проекте мы сосредоточены на предсказаниях, поэтому проблема мультиколлинеарности, если она и есть, для нас не страшна, на предсказаниях она не сказывается (она влияет на значения коэффициентов логистической регрессии `.coef_` (это, кстати, <a href="https://betterdatascience.com/feature-importance-python/">о важности признаков</a>), для скоррелированных признаков они смещены, поэтому их интерпретация становится невозможной, но для нас сейчас это не имеет значения). Для отбора признаков вместо корреляции можно использовать функции feature selection уже после обучения моделей, то есть когда она с ними познакомилась и сама нам уже говорит, какие для нее важны, а какие не очень.
- Рекомендую также тебе посмотреть метод `sns.pairplot` с параметром `hue`, в который мы передаем целевой признак.
</div>
        

Обратимся к вики, чтобы разобраться, что же это такое

Мультиколлинеарность (multicollinearity) — в эконометрике (регрессионный анализ) — наличие линейной зависимости между объясняющими переменными (факторами) регрессионной модели. При этом различают полную коллинеарность, которая означает наличие функциональной (тождественной) линейной зависимости и частичную или просто мультиколлинеарность — наличие сильной корреляции между факторами.

Собственно говоря, как я понял, это сильная взаимосвязь параметров (в нашем случае столбцов), вероятно, можно избавиться от одного для дальнейшего анализа, поскольку все взаимосвязи в другими параметрами будут индентичны, я могу без угрызений совести выкинуть одну из двух фич (минуты или количество звонков). Но делать так не хочется, мало ли что-то сломается в последствии, хотя уверен, что в дальнейшем с этими параметрами не работаю от слова совсем, только загоняю их в признаки на вход модели и от того, что снесу один ничего не поменяется :)) 

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"></a></h2>

<b>Все отлично!👍:  </b> 
    
Да, что мы можем сделать - это или удалить один из признаков, коррелирующих с другим, или же создать новые признаки на основе выделения факторов с помощью метода главных компонент (Principal Main Component).    

</div>

In [3]:
data.corr(method='spearman')

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.978684,0.155032,0.253886,0.160549
minutes,0.978684,1.000000,0.153784,0.248818,0.159991
messages,0.155032,0.153784,1.000000,0.141999,0.106537
mb_used,0.253886,0.248818,0.141999,1.000000,0.154989
is_ultra,0.160549,0.159991,0.106537,0.154989,1.000000


Да, что и требовалось доказать, для нахождения закономерности с другим столбцами они взаимозаменяемы и это лишняя трата ресурсов вычислительных.

## Разбейте данные на выборки

Разобьем исходные данные на три выборки: обучающую, валидационную и тестовую

В sklearn для этого предусмотрена функция train_test_split. Она разбивает любой датасет на две выборки в зависимости от компонента test_size. Чтобы разбить на три выборки произведем разбиение дважды, сначала разобьем исходные дата сет на обучающую выборку, куда уйдет 60% данных, и другую часть, куда уйдут 40%. Затем разобьем эту часть пополам и половину запишем под валидационную, а другую под тестовую, таким образом у нас получиться 60% обучающей + 20% валидационной + 20% тестовой.

In [4]:
data_train, data_40 = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_40, test_size=0.5, random_state=12345)

### Вывод 
Получили три выборки с разделение 60-20-20 дважды поделив их на две части.

In [5]:
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(1928, 5)
(643, 5)
(643, 5)


Выведем размерность наших разбиений, как видим, все вышло как надо! 

<div class="alert alert-warning">
    <h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b>
    
С разбиением все в порядке. Желательно, чтобы размеры полученных наборов были напечатаны. Иногда, это помогает избежать ошибок.
 </div>

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Супер!

## Исследуйте модели

Прежде чем исследовать модели, необходимо определиться с целевым признаком, для этого нам понадобится бизнес-требование, которое гласит: "Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра»."

Главная задача - это на основе некоторых входных признаков научиться прогназировать, кому более подходит тариф Ультра, а кому нет (данные по нему храняться в столбце "is_ultra", состоящего из 1 и 0 (кстати, столбец разумнее сделать булевым)). Отсюда можно сделать вывод, что перед нами стоит задача бинарной классификации. А целевой признак и есть данный столбец "is_ultra".

Таким образом, нам необходимо создать модель и обучить ее предсказывать данные данного столбца, причем с наименьшей погрешностью. Чтобы минимизировать погрешности, будем перебирать разные модели машинного обучения, которые проходили в данном спринте, а именно:
- Дерево решений
- Случайный лес
- Логистическая регрессия

Мы не просто попробуем разные модели, а попробуем повзаимодействовать с их гиперпараметрами, для достижения более точного результата с минимальными погрешностями.

Так как наборы признаков будут одинаковы для каждой модели, выделим наши features&target для обучения прямо сейчас:

In [6]:
#data['is_ultra'].unique()
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

### Модель Дерево решений

In [7]:
accuracy_list = [] # список для значений точности

for depth in range(2, 20): # реализуем цикл для подстановки значений в гиперпараметр глубины дерева
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) # определяем вид модели и ее гиперпараметры
    
    model_tree.fit(features_train, target_train) # обучаем модель на обучающем наборе данных
    
    predictions_tree = model_tree.predict(features_valid) # проверяем модель на валидационном наборе данных

    accuracy = round(accuracy_score(target_valid, predictions_tree), 5) # округлим до 5 знаков для читаемости
    accuracy_list.append(accuracy) # добавляем в список точности
    
print(accuracy_list)
print()    
print('Максимальная доля правильных ответов', max(accuracy_list), 'при max_depth =', np.argmax(accuracy_list)+2)

best_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=np.argmax(accuracy_list)+2)# сохраним лучший результат

[0.78227, 0.78538, 0.77916, 0.77916, 0.78383, 0.78227, 0.77916, 0.78227, 0.77449, 0.76205, 0.76205, 0.75583, 0.75894, 0.7465, 0.73406, 0.73561, 0.73095, 0.72784]

Максимальная доля правильных ответов 0.78538 при max_depth = 3


Лучший результат достигается при глубине дерева равной трем. Получили точность около 78,5%.

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.1 <a class="tocSkip"> </h2>    

Некоторые блоки кода не работают. Посмотри, пожалуйста, что пошло не так. Перед отправкой проекта стоит проверять работоспособность кода - это можно сделать, нажав на панели `Jupiter Hub Kernel` и `Restart & Run All`
      
    
Жду от тебя новую версию проекта.    

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> 

### Модель Случайный лес

В модели Случайный лес будут использоваться два цикла (как и в задании с тренажером), один под глубину дерева, другой под количество деревьев. Двойной цикл не лучшее с точки зрения оптимизации, но что поделать!  

In [8]:
# соберем значения accuracy и гиперпараметров в списки, из которых потом создадим датафрейм
accuracy_list = []
max_depth_list = []
n_estim_list = []

for depth in range(2, 20):
    for est in range(10, 51, 10): # установим шаг 10, потому что ждать 2 минуты результата - можно взгрустнуть:C
        model_rf = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est) # определяем вид модели и ее гиперпараметры
        
        model_rf.fit(features_train, target_train) # обучаем модель на обучающем наборе данных
        
        predictions_rf = model_rf.predict(features_valid) # проверяем модель на валидационном наборе данных
        
        accuracy = round(accuracy_score(target_valid, predictions_rf), 5) # округлим до 5 знаков для читаемости
        
        accuracy_list.append(accuracy) 
        max_depth_list.append(depth)
        n_estim_list.append(est)
        
# собираем все списки в один
result_list = [max_depth_list, n_estim_list, accuracy_list]

# создаем датафрейм
data_random_forest = pd.DataFrame({'max_depth':max_depth_list, 'n_estimators':n_estim_list, 'accuracy':accuracy_list})

# выводим на экран строку с наибольшей точностью
display(data_random_forest[data_random_forest['accuracy']==data_random_forest['accuracy'].max()])

best_forest_model = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=40) # сохраним по результатам прогона

,max_depth,n_estimators,accuracy
33,8,40,0.80871


Самый лучший результат получили при глубине дерева 15 и количестве деревьев 50. Доля правильных ответов составляет более 80%.

### Модель Логистическая регрессия

In [9]:
for solve in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    model = LogisticRegression(random_state=12345, solver=solve)

    model.fit(features_train, target_train)

    predictions = model.predict(features_valid)

    accuracy_lr = round(accuracy_score(target_valid, predictions), 5)
    print(accuracy_lr)
    
best_reg_model = LogisticRegression(random_state=12345, solver='newton-cg')

0.75583
0.71073
0.70918
0.70607
0.70607


Нашел в документации различные возможные параметры гиперпараметра solver, это вероятно выбранный алгоритм для логистической регрессии, пока что не разобрался наверняка как он работает, но для разнообразия перебрал его, и как мы видим, выбил этим самым результат >75%. Правда красные варнинги очень настораживают ...

<div class="alert alert-warning">
    <h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 
    
Алгоритм логистической регрессии не сходится. Для логистической регрессии, когда алгоритм не сходится, необходима стандартизация данных, но на качество предсказаний это никак не влияет. Также стоит попробовать увеличить параметр `max_iter`, пока не пропадут предупреждения.
        
У моделей на основе деревьв принцип принятия решения завязан на относительности значений признаков, а она при стандартизации воздействию не подвергается. 


По ссылкам можно узнать, когда еще стандартизация крайне важна: <a href = 'https://www.listendata.com/2017/04/how-to-standardize-variable-in-regression.html'>ссылка 1</a>, <a href = 'https://www.statlect.com/fundamentals-of-statistics/linear-regression-with-standardized-variables'>ссылка 2</a>.

Еще преимущество, которое дает стандартизация для логистической и линейной регрессии, это то, что мы можем по величине полученных коэффициентов судить о важности соотвествующих признаков (больше коэффициент - больше значимость, важность необходима для регрессий с регуляризацией). Без стандартизации такая интерпретация невозможна, так как каждый коэффициент в себя включает еще и элемент балансировки масштаба своего признака.
        
С другой стороны, применяя стандартизацию к признакам, нужно быть осторожным: не допустить утечку в данных. Стандартизацию следует применить к тренировочной выборке и распространить ее на валидационную и тестовую. Для выборки только с численными признаками это выглядит так:

    Код ревьюера:
```python   

# С валидационной выборкой    
scaler = StandardScaler()
scaler.fit(X_train) 
    
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)



<br/>    
<div class="alert alert-info">
<h2> Комментарий студента V.2: <a class="tocSkip"> </h2>

Обязательно разберусь со стандартизацией, перед заливкой на гитхаб, но сейчас дедлайны горят, стараюсь максимально оперативно нагнать все свои дедлайны, спасибо большое за информацию, поковыряю, даю слово! 
</div> 

<br/>    

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b>  Пожалуйста)

### Вывод по главе: 
Таким образом мы можем выделить наиболее удачные модели, а именно дерево решений с глубиной 3, случайный лес с глубиной 15 и количеством деревьев равным 50, а так же логистическую регрессию с параметром solver='newton-cg'.

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"></a></h2>

<b>Все отлично!👍:</b>

Модели исследованы корректно:
    
 - исследовано более 5 значений гиперпараметра
 - модели обучены на обучающем наборе
 - получена оценка качества на валидационном наборе
 - перебор гиперпараметров осуществляется в цикле
 - есть выводы   
</div>

## Проверьте модель на тестовой выборке

In [10]:
best_tree_model.fit(features_train, target_train) 
test_predictions = best_tree_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели дерева решений на тестовой выборке',accuracy)

best_forest_model.fit(features_train, target_train) 
test_predictions = best_forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели случайного леса на тестовой выборке',accuracy)

best_reg_model.fit(features_train, target_train) 
test_predictions = best_reg_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели логистической регресиии на тестовой выборке',accuracy)

Точность модели дерева решений на тестовой выборке 0.7791601866251944
Точность модели случайного леса на тестовой выборке 0.7962674961119751
Точность модели логистической регресиии на тестовой выборке 0.7387247278382582


Снова варнинги:C Немного костыльно выглядит, снова пришлось прописывать обучение, так как я записывал отдельно лучшие модели, но вроде работает, лучшая точность у Случайного леса, худшая у логистической регрессии.

<div class="alert alert-warning">
    <h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 
        
Предупреждения можно отключить, добавив в ячейку с импортами:
```python
import warnings
warnings.filterwarnings('ignore')
``` 


<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> 

<div class="alert alert-warning">
    <h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b>
    
На тестовой выборке получено хорошее качество (она у нас играет роль отложенной, holdout), но знай, что тестирование положено проводить для одной лучшей модели. На предыдущем шаге мы должны были настроить модели и выбрать одну наилучшую, опираясь на метрики данные заказчиком (это может быть качество, время обучения, скорость предсказания и т.д.). Тестированием мы моделируем работу модели на новых незнакомых ей данных, которые ни разу не использовались ни при тренировке, ни при валидации, и проверяем, не словили ли мы переобучение. А эти данные могут быть смещенными, с выбросами и т.д. То есть по таким данным некорректно заново переопределять модель-победитель.  Советую тебе статью, рассматривающую разные способы валидации моделей машинного обучения: https://towardsdatascience.com/validating-your-machine-learning-model-25b4c8643fb7
        
Метрики качества на тестовой выборке для всех моделей можно вычислить только с целью исследования их смещения относительно аналогичных метрик на валидационной выборке. Но это не означает, что модель-победитель должна выбираться исходя из сравнения метрик, полученных на тестовой выборке.        

## (бонус) Проверьте модели на адекватность

Ответ на данный раздел я нашел в глубинах интернета, хотя как глубинах, минут 10, в целом понял (хотя узнал, что такоее среднее гармоническое), так что теперь смогу провернуть это для любой задачи классификации.

Оценить адекватность модели можно используя несколько метрик:

- accuracy - точность совпадения прогнозов с правильными ответами.
- precision - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправльными предсказаниями, указанные как 1(FP).
- recall - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправльными предсказаниями, указанные как 0(FN).

Для оценки адеватности используется F-мера - среднее гармоническое между precision и recall. Если хотя бы один из параметров близок к нулю, то и F-мера стремится к 0. Если оба стремятся к 1, то F-мера тоже стремится к 1.

In [11]:
test_predictions = best_forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
precision = precision_score(target_test, test_predictions)
recall = recall_score(target_test, test_predictions)
print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

f_score = f1_score(target_test, test_predictions)
print('F-мера =', f_score)

Accuracy = 0.7962674961119751 Precision = 0.75 Recall = 0.5320197044334976
F-мера = 0.622478386167147


### Вывод

Адекватность модели оценивается по нескольким параметрам. Для модели классификации это accuracy, precision и recall. Качество модели лучше всего отражают precision и recall и эти метрики складываются в F-мера как единую оценку модели.

Модель получилась среднего качества. И для таких задач она адекватна.

<div class="alert alert-success">
<h2> Комментарий ревьюера v.2 <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> 
    
Этап тестирования рекомендуется дополнять введением в работу baseline модели. Сравнивая результаты более сложных моделей с самой наивной, мы можем делать выводы о том, насколько далеко они вообще продвинулись в качестве предсказаний. Иногда может случиться такое, что константная модель оказывается не сильно хуже по качеству. Это может говорить либо о том, что еще есть смысл продолжить поиски "той самой" модели, либо структура данных такова, что ничего лучше константоной в принципе не найти, тогда в использовании более сложных ресурсозатратных моделей нет особого смысла. Создавать константные модели можно либо вручную, либо воспользоваться готовым алгоритмом <a href='https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html'>DummyClassifier</a> из пакета `sklearn`. В нашем случае подойдет модель DummyClassifier со стратегией `most_frequent`. Рекомендую также прочитать  <a href='https://progler.ru/blog/ml-klassifikatory-pustyshki-s-ispolzovaniem-sklearn'>статью</a>.

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.2 <a class="tocSkip"> </h2>    
    
  
У меня сложилось хорошее впечатление о проекте. Молодец! Изучены все параметры, построено несколько моделей классификации и оценено их качество. Осмысленная аналитика и дельная модельная работа - многое удалось как надо)
    
Отмечу отдельные положительные моменты проекта🙂:
    
- тебе удалось добиться очень хорошего качества, поздравляю!  
- при обучении моделей использована валидационная выборка и подбор гиперпараметров.    
    
Проект может быть зачтен, но я его отправлю назад, чтобы у тебя была возможность задать вопросы и внести правки, при желании. Однако, ты можешь просто вернуть проект в таком же виде и я его зачту.
     
</div>   


<div class="alert alert-info">
<h2> Комментарий студента V.2: <a class="tocSkip"> </h2>
    
Спасибо большое тебе за твою работу и за дополнительные материалы по проекту, обязательно еще к ним обращусь!
</div> 

<br/>    

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.3 <a class="tocSkip"> </h2>    
    
     
  
Принимаю твой проект)
    
Могу посоветовать посмотреть видеоразбор <a href="https://www.youtube.com/watch?v=YTLDATQLK6w">Как обучить модель по шагам в Data Science?</a>
    
Рекомендую тебе ознакомиться с  `GridSearchCV` для подбора гиперпараметров, это избавит тебя от написания циклов. Полезные ссылки: https://vc.ru/ml/147132-kak-avtomaticheski-podobrat-parametry-dlya-modeli-mashinnogo-obucheniya-ispolzuem-gridsearchcv
        
https://newtechaudit.ru/gridsearchcv-pomoshhnik-v-vybore/        
    
    
В этом проекте все шаги отработаны. С интересом читал твои размышления, код и выводы. Цель нашего проекта достигнута. Рад был тебе помочь)
    
Преподаватель в Slack будет рад помочь, если у тебя появятся вопросы.
  
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
